In [388]:
#!/usr/bin/env python
#example
import grpc
import rospy
from concurrent import futures
# import the publisher and subscriber node
from ros_bridge import RosBridge
# import the gRPC generated classes
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2, robot_server_pb2_grpc

class Client():

    def __init__(self, server_port):
        self.channel = grpc.insecure_channel('[::]:' + repr(server_port))
        self.robot_server_stub = robot_server_pb2_grpc.RobotServerStub(self.channel)

    def set_state(self, state):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.SetState(robot_server_pb2.State(state = state), timeout = 60)
        return msg.success

    def set_state_msg(self, state_msg):
        msg = self.robot_server_stub.SetState(state_msg, timeout = 60)
        return msg.success

    def get_state(self,):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg.state
        else:
            raise Exception('Error while retrieving state')

    def get_state_msg(self,):
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')

    def send_action(self, action):
        msg = self.robot_server_stub.SendAction(robot_server_pb2.Action(action = action), timeout = 20 )
        return msg.success

    def send_action_get_state(self, action):
        msg = self.robot_server_stub.SendActionGetState(robot_server_pb2.Action(action = action), timeout = 20 )
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')


In [389]:
rospy.loginfo('Starting Example Robot Server...')
server_port = 50051
client = Client(server_port)
channel = grpc.insecure_channel('[::]:' + repr(server_port))
robot_server_stub = robot_server_pb2_grpc.RobotServerStub(channel)

In [15]:
state = client.get_state()
print("state: ", state)

state:  [0.2546505331993103, -0.7181203961372375, -1.1508787870407104, 0.7403168678283691, 0.004182456061244011, 0.1120019182562828, -0.21588945388793945, -0.2701323926448822, -0.001399481319822371, 0.0015176599845290184, -0.005273917689919472, 0.0025401096791028976, 7.659690709260758e-06, -0.0001162272019428201, 0.00024791431496851146, 0.00015039996651466936]


In [6]:
get_state_msg =client.get_state_msg()
print(get_state_msg)
print("state: ",get_state_msg.state)
print("float_params: ", get_state_msg.float_params)
print("string_params: ", get_state_msg.string_params)
print("state_dict: ", get_state_msg.state_dict)


state: -0.04027082398533821
state: -0.6972288489341736
state: -0.7374358773231506
state: 0.9174924492835999
state: -0.04152611643075943
state: 0.012637495994567871
state: -0.11190915107727051
state: -0.13079914450645447
state: 0.005225419532507658
state: 0.0012128078378736973
state: 0.0059148347936570644
state: -0.0009095278801396489
state: 9.332344461654429e-07
state: -0.0008182747405953705
state: -0.0004433365538716316
state: -7.56235203880351e-06
state_dict {
  key: "front_left_steering_joint_position"
  value: -0.04027082398533821
}
state_dict {
  key: "front_left_steering_joint_velocity"
  value: 0.005225419532507658
}
state_dict {
  key: "front_left_wheel_joint_position"
  value: -0.04152611643075943
}
state_dict {
  key: "front_left_wheel_joint_velocity"
  value: 9.332344461654429e-07
}
state_dict {
  key: "front_right_steering_joint_position"
  value: -0.6972288489341736
}
state_dict {
  key: "front_right_steering_joint_velocity"
  value: 0.0012128078378736973
}
state_dict {
  

### set state

In [211]:
import numpy as np
link_names = [ 'front_left_steering_link', 'front_right_steering_link', 'rear_left_steering_link', 'rear_right_steering_link', 'front_left_wheel_link', 'front_right_wheel_link', 'rear_left_wheel_link', 'rear_right_wheel_link']
link_state = dict.fromkeys(link_names, np.deg2rad(0))
print(link_state)

{'front_left_steering_link': 0.0, 'front_right_steering_link': 0.0, 'rear_left_steering_link': 0.0, 'rear_right_steering_link': 0.0, 'front_left_wheel_link': 0.0, 'front_right_wheel_link': 0.0, 'rear_left_wheel_link': 0.0, 'rear_right_wheel_link': 0.0}


In [49]:
links = {'front_left_steering_link': [1, 1],
	                    'front_right_steering_link': [1, -1],
	                    'rear_left_steering_link': [-1, 1],
	                    'rear_right_steering_link': [-1, -1] }
links

{'front_left_steering_link': [1, 1],
 'front_right_steering_link': [1, -1],
 'rear_left_steering_link': [-1, 1],
 'rear_right_steering_link': [-1, -1]}

In [254]:

# state_msg = robot_server_pb2.State(state_dict = client.get_state_msg().state_dict)
# link_state = {'rear_right_steering_link': 0}
link_state_msg = robot_server_pb2.State(state_dict=link_state)
print(type(link_state_msg))
client.set_state_msg(link_state_msg)


<class 'robot_server_pb2.State'>


True

### send action

In [415]:
action = client.send_action([100, -500, 40, -500])
action

True

In [277]:
['front_left_steering_joint', 'front_left_wheel_joint', 'front_right_steering_joint', 'front_right_wheel_joint', 'rear_left_steering_joint', 'rear_left_wheel_joint', 'rear_right_steering_joint', 'rear_right_wheel_joint']

False

In [ ]:
send_action_get_state = client.send_action_get_state([1, -.2])
send_action_get_state

In [271]:
new_state = {'pos_x': 1.5, 'pos_y': 2}
new_state
new_state_msg = robot_server_pb2.State(state_dict = new_state)
new_state_msg
robot_server_stub.SetState(new_state_msg)

success: true

In [ ]:
# State must be dict see rosbridge
new_state = [1, 1, 1, 1]
new_state
new_state_msg = robot_server_pb2.State(state = new_state)
new_state_msg
robot_server_stub.SetState(new_state_msg)